To run this notebook, **bnci_utils.py** is necessary. Before running this, preprocessing is necessary which can
be done using the

Make sure that all the dataset files are located in "dataset" folder
and that it does not contain any other files (except \_\_init\_\_.py)

In [2]:
import os
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
from tensorflow.python.keras import Input, Model
import nengo
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, \
    Conv3D, MaxPooling2D, Conv1D, MaxPooling1D
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K
import pandas as pd
from sklearn import metrics

import bnci_utils as utils

In [3]:
# All the datasets that can be run with this notebook
#   Entire dataset - all data
#   Female subjects - data from female subjects
#   Male subjects - data from male subjects
datasets = {
    'entire_dataset': 'entire_dataset.npz',
    'female_subjects': 'dataset_female_gender.npz',
    'male_subjects': 'dataset_male_gender.npz'
}

dataset_path = os.path.join('dataset_result', datasets['male_subjects'])

data_output_folder = 'entire_dataset_output_cnn' # output path for statistics from the simulation
iteration_data_file_name = 'cnn_exp_10_fold_male_subj.xlsx' # file name of excel file with data from each iteration
iteration_stats_file_name = 'cnn_exp_10_fold_male_subj_stats.xlsx' # file name for statistics from the simulation (i.e
                                                                   # max and average accuracy, max and average recall...)


# List of tested models - default CNN is the originally tested network, p300_exp_cnn is the CNN from the P300 experiment
models = {
    'default_cnn': utils.cnn_model,
    'p300_exp_cnn': utils.original_p300_model
}

# Model function to create the model for simulation
model_fn = models['default_cnn']

In [4]:
# Get features and labels
features, labels = utils.load_dataset(dataset_path)

f'Features shape: {features.shape}, labels shape: {labels.shape}'

'Features shape: (1296, 14, 36, 10), labels shape: (1296,)'

In [5]:
# Check if the dataset is balanced
yes = labels[labels == 'yes']
no = labels[labels == 'no']

f'yes: {yes.shape} ({(yes.shape[0]/labels.shape[0]) * 100}%), no: {no.shape} ({(no.shape[0]/labels.shape[0]) * 100}%)'

'yes: (672,) (51.85185185185185%), no: (624,) (48.148148148148145%)'

In [6]:
# Reshape the dataset
features, labels = utils.reshape_dataset(features, labels)
f'Features shape: {features.shape}, labels shape: {labels.shape}'

'Features shape: (1296, 1, 5040), labels shape: (1296, 1, 2)'

In [7]:
# Set seed for consistency
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=seed, shuffle=True)

f'x_train shape: {x_train.shape}, y_train shape: {y_train.shape}, ' \
f'x_test shape: {x_test.shape}, y_test shape: {y_test.shape}'

'x_train shape: (972, 1, 5040), y_train shape: (972, 1, 2), x_test shape: (324, 1, 5040), y_test shape: (324, 1, 2)'

In [9]:
params_output_path = 'cnn_all_samples_nengo_params'
os.makedirs(params_output_path, exist_ok=True)

utils.cnn_model(seed=seed).summary() # Print the model

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 14, 360, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 360, 32)       832       
_________________________________________________________________
dropout (Dropout)            (None, 14, 360, 32)       0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 7, 180, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 178, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 178, 64)        0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 2, 89, 64)         0     

In [10]:
ann, snn = [], [] # arrays that will contain data from each iteration for the analog and spiking network

num_iterations = 10 # number of iterations in the cross-validation (10)
iteration = 1 # number of current iteration

for train, valid in KFold(n_splits=num_iterations).split(x_train): # perform K-Fold CV
    print('Current iteration: ', iteration)
    x_train_curr, y_train_curr = x_train[train], y_train[train] # get current training data
    x_val_curr, y_val_curr = x_train[valid], y_train[valid] # get current validation data

    params_path = os.path.join(params_output_path, f'params_{iteration}') # configure path for parameters

    model = model_fn(seed=seed) # create the model

    # run ann
    ann_result = utils.run_ann(model=model,
                               train=(x_train_curr, y_train_curr),
                               valid=(x_val_curr, y_val_curr),
                               test=(x_test, y_test),
                               optimizer=keras.optimizers.Adam(),
                               loss=keras.losses.BinaryCrossentropy(),
                               params_save_path=params_path,
                               iteration=iteration,
                               callbacks=[EarlyStopping(patience=8, restore_best_weights=True, verbose=1)],
                               num_epochs=30
                         )

    # run snn
    snn_result = utils.run_snn(model,
                               x_test, y_test,
                               params_load_path=params_path,
                               iteration=iteration
                               )

    ann.append(ann_result)
    snn.append(snn_result)
    iteration += 1

    K.clear_session() # clear session and delete model since it sometimes causes memory leaks
    del model

Current iteration:  1
|                     Building network (0%)                    | ETA:  --:--:--

c:\dev\anaconda\envs\conda_tutorial\lib\site-packages\nengo_dl\converter.py:324: UserWarning: Layer type <class 'tensorflow.python.keras.layers.core.Dropout'> does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\conda_tutorial\lib\site-packages\nengo_dl\converter.py:586: UserWarning: Activation type <function softmax_v2 at 0x000002B021647F70> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(


Build finished in 0:00:01                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
|             Constructing graph: build stage (0%)             | ETA:  --:--:--

c:\dev\anaconda\envs\conda_tutorial\lib\site-packages\nengo_dl\simulator.py:1927: UserWarning: Number of elements in input data (98) is not evenly divisible by Simulator.minibatch_size (16); input data will be truncated.
  warnings.warn(
c:\dev\anaconda\envs\conda_tutorial\lib\site-packages\nengo_dl\simulator.py:1927: UserWarning: Number of elements in input data (874) is not evenly divisible by Simulator.minibatch_size (16); input data will be truncated.
  warnings.warn(


54/54 [==============================] - 9s 63ms/step - loss: 1.0161 - probe_loss: 1.0161 - probe_accuracy: 0.5415 - val_loss: 0.6941 - val_probe_loss: 0.6941 - val_probe_accuracy: 0.4896
Epoch 2/30
54/54 [==============================] - 1s 23ms/step - loss: 0.6929 - probe_loss: 0.6929 - probe_accuracy: 0.5153 - val_loss: 0.6943 - val_probe_loss: 0.6943 - val_probe_accuracy: 0.4896
Epoch 3/30
54/54 [==============================] - 1s 23ms/step - loss: 0.6953 - probe_loss: 0.6953 - probe_accuracy: 0.5294 - val_loss: 0.6853 - val_probe_loss: 0.6853 - val_probe_accuracy: 0.5104
Epoch 4/30
54/54 [==============================] - 1s 23ms/step - loss: 0.6954 - probe_loss: 0.6954 - probe_accuracy: 0.4637 - val_loss: 0.6990 - val_probe_loss: 0.6990 - val_probe_accuracy: 0.4792
Epoch 5/30
54/54 [==============================] - 1s 23ms/step - loss: 0.6948 - probe_loss: 0.6948 - probe_accuracy: 0.5018 - val_loss: 0.6964 - val_probe_loss: 0.6964 - val_probe_accuracy: 0.4896
Epoch 6/30
54/54

...

10. CNN: accuracy = 54.6875%, precision = 0.5526315789473685, recall = 0.8497109826589595, f1 = 0.6697038724373576
Confusion matrix:
[[ 28 119]
 [ 26 147]]
|                     Building network (0%)                    | ETA:  --:--:--

c:\dev\anaconda\envs\conda_tutorial\lib\site-packages\nengo_dl\converter.py:324: UserWarning: Layer type <class 'tensorflow.python.keras.layers.core.Dropout'> does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\conda_tutorial\lib\site-packages\nengo_dl\converter.py:586: UserWarning: Activation type <function softmax_v2 at 0x000002B021647F70> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(


Build finished in 0:00:01                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Constructing graph: pre-build stage finished in 0:00:00                        

c:\dev\anaconda\envs\conda_tutorial\lib\site-packages\nengo_dl\simulator.py:1765: UserWarning: Number of elements (1) in ['ndarray'] does not match number of Nodes (4); consider using an explicit input dictionary in this case, so that the assignment of data to objects is unambiguous.
  warnings.warn(


10. CNN (SNN conversion): accuracy = 56.56250000000001%, precision = 0.5658914728682171, recall = 0.8439306358381503, f1 = 0.6774941995359629
Confusion matrix:
[[ 35 112]
 [ 27 146]]


In [11]:
# Create data dictionary for pandas dataframe
df = utils.create_data_df(ann, snn, num_iterations)

df

,iterations,ann_accuracy,ann_precision,ann_recall,ann_f1,snn_accuracy,snn_precision,snn_recall,snn_f1
0,1,0.459375,0.500000,0.011561,0.022599,0.450000,0.466667,0.121387,0.192661
1,2,0.534375,0.544776,0.843931,0.662132,0.428125,0.250000,0.028902,0.051813
2,3,0.537500,0.554113,0.739884,0.633663,0.503125,0.589744,0.265896,0.366534
3,4,0.531250,0.573248,0.520231,0.545455,0.496875,0.536145,0.514451,0.525074
4,5,0.543750,0.542320,1.000000,0.703252,0.490625,0.519841,0.757225,0.616471
5,6,0.531250,0.542751,0.843931,0.660633,0.543750,0.546713,0.913295,0.683983
6,7,0.540625,0.540625,1.000000,0.701826,0.518750,0.544601,0.670520,0.601036
7,8,0.540625,0.540625,1.000000,0.701826,0.509375,0.531746,0.774566,0.630588
8,9,0.540625,0.540625,1.000000,0.701826,0.512500,0.530249,0.861272,0.656388
9,10,0.546875,0.552632,0.849711,0.669704,0.565625,0.565891,0.843931,0.677494


In [12]:
os.makedirs(data_output_folder, exist_ok=True)

# Save the dataframe to excel
df.to_excel(os.path.join(data_output_folder, iteration_data_file_name))

# Save path for the P300 model
# df.to_excel(os.path.join(data_output_folder, 'cnn_p300_model_10_fold_entire_dataset.xlsx'))

'Statistics for iterations successfully saved.'

'Statistics for iterations successfully saved.'

In [13]:
# Create statistics such as maximums and averages for each metric
df_stats = utils.create_stats_df(df)

df_stats

,models,average_accuracy,max_accuracy,accuracy_std,average_precision,max_precision,average_recall,max_recall,average_f1,max_f1
0,ann,0.530625,0.546875,0.025550,0.543171,0.573248,0.780925,1.000000,0.600291,0.703252
1,snn,0.501875,0.565625,0.040214,0.508160,0.589744,0.575145,0.913295,0.500204,0.683983


In [14]:
# Create dataframe for statistics and save it as excel file
df_stats.to_excel(os.path.join(data_output_folder, iteration_stats_file_name))

# Save path for the P300 model
# df_stats.to_excel(os.path.join(data_output_folder, 'cnn_p300_model_10_fold_entire_dataset_stats.xlsx'))

'File with statistics successfully saved.'

'File with statistics successfully saved.'

In [15]:
# Print confusion matrices for ANN and SNN in each iteration
utils.print_confusion_matrices(ann, snn)


Confusion matrices for the ANN:
[[145   2]
 [171   2]] 

[[ 25 122]
 [ 27 146]] 

[[ 44 103]
 [ 45 128]] 

[[80 67]
 [83 90]] 

[[  1 146]
 [  0 173]] 

[[ 24 123]
 [ 27 146]] 

[[  0 147]
 [  0 173]] 

[[  0 147]
 [  0 173]] 

[[  0 147]
 [  0 173]] 

[[ 28 119]
 [ 26 147]] 

Confusion matrices for the SNN
[[123  24]
 [152  21]] 

[[132  15]
 [168   5]] 

[[115  32]
 [127  46]] 

[[70 77]
 [84 89]] 

[[ 26 121]
 [ 42 131]] 

[[ 16 131]
 [ 15 158]] 

[[ 50  97]
 [ 57 116]] 

[[ 29 118]
 [ 39 134]] 

[[ 15 132]
 [ 24 149]] 

[[ 35 112]
 [ 27 146]] 

